In [1]:
"""
this file will specify the config, and then call benchmark.py to calculate the brain score metrics.
this will also append the results to a common file.
"""
import sys
sys.path.append('/home/akazemi3/Desktoparchvision/archvision')
from omegaconf import OmegaConf


cfg_dict = {
    

"model": {
    "norm": "instance",
    "weights_init": "kaiming",
    "nonlin": "relu",
    "wavelet_type": "bior",
    "wavelet_layers": 2,
    "conv_in_channels": "none",
    "layers": [
        {
            "channels": 128,
            "kernel_size": 5,
            "pooling": "max",
            "pool_kernel_size": 2
        },
        {
            "channels": 256,
            "kernel_size": 3,
            "pooling": "avg",
            "pool_kernel_size": 2
        },
        {
            "channels": 256,
            "kernel_size": 3,
            "pooling": "none",
            "pool_kernel_size": "none"
        }
    ]
    },
    
"voxel_set": "OTC",
"eval_metric": "srpr",
"benchmark_data_type": "fMRI",
"log_expdata": True,
"log_dir": "logs/",
"exp_name": "test"
}



cfg = OmegaConf.create(cfg_dict)
config = OmegaConf.from_cli()
cfg = OmegaConf.merge(cfg, config)

In [2]:
import torch
import torchvision.transforms as transforms
from models.backbone import VisionModel

sys.path.append('/home/akazemi3/Desktop/DeepJuiceDev')
from deepjuice import * # the juice
from benchmarks.nsd_data import NSDBenchmark

devices = {'device': 'cuda', 'output_device': 'cpu'}


preprocess = transforms.Compose([
    transforms.Resize(256),             
    transforms.CenterCrop(224),         
    transforms.ToTensor(),             
    transforms.Normalize(             
        mean=[0.485, 0.456, 0.406],     
        std=[0.229, 0.224, 0.225])
])

model = VisionModel(cfg, device=devices['device'])
print(model(torch.Tensor(1,3,224,224)).shape) # test with sample image


benchmark = NSDBenchmark() #load brain data benchmark
dataloader = get_data_loader(benchmark.image_paths, preprocess)

extractor = FeatureExtractor(model, dataloader, flatten=True,
                             batch_progress=True, **devices)

Loading deepjuice benchmarks...
torch.Size([1, 256, 18, 18])
Now loading the NSD_shared1000 image set and the EVC-OTC voxel set...
Extracting sample feature_maps with torchinfo.
FeatureExtractor Handle for VisionModel
  15 feature maps (-8 duplicates); 1000 inputs
  Memory required for full extraction: 64.833 GB
  Memory usage limiting device set to: cpu
  Memory usage limit currently set to: 90.380 GB
  1 batch(es) required for current memory limit 
   Batch-001: 15 feature maps; 64.833 GB
